In [68]:
import pandas as pd
import folium

In [69]:
centre_latlng = [40.738, -73.98]

In [70]:
folium_map = folium.Map(location=centre_latlng,
                        zoom_start=5,
                        tiles="CartoDB dark_matter")

In [71]:
folium.CircleMarker(location=centre_latlng,fill=True).add_to(folium_map)
folium_map

In [72]:
bike_data = pd.read_csv("201811-citibike-tripdata.csv")

In [73]:
bike_data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,254,2018-11-01 00:00:12.9100,2018-11-01 00:04:27.3870,3522.0,37 St & 24 Ave,40.770148,-73.912094,3513.0,21 St & Hoyt Ave S,40.774645,-73.923706,34117,Subscriber,1992,1
1,833,2018-11-01 00:00:16.3440,2018-11-01 00:14:09.4320,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,251.0,Mott St & Prince St,40.723180,-73.994800,32569,Subscriber,1986,2
2,683,2018-11-01 00:00:17.6630,2018-11-01 00:11:40.9170,3405.0,5 St & 6 Ave,40.670484,-73.982090,3584.0,Eastern Pkwy & Franklin Ave,40.670777,-73.957680,15795,Subscriber,1985,1
3,1840,2018-11-01 00:00:18.7970,2018-11-01 00:30:59.3050,3150.0,E 85 St & York Ave,40.775369,-73.948034,297.0,E 15 St & 3 Ave,40.734232,-73.986923,33290,Subscriber,1994,1
4,1212,2018-11-01 00:00:47.3230,2018-11-01 00:20:59.8840,325.0,E 19 St & 3 Ave,40.736245,-73.984738,212.0,W 16 St & The High Line,40.743349,-74.006818,17411,Subscriber,1989,1


In [74]:
bike_data["starttime"] = pd.to_datetime(bike_data["starttime"])
bike_data["stoptime"] = pd.to_datetime(bike_data["stoptime"])
bike_data["hour"] = bike_data["starttime"].map(lambda x: x.hour)

In [75]:
selected_hour = 10

In [76]:
def get_trip_counts(selected_hour):
    locations = bike_data.groupby("start station id").first()
    locations = locations.loc[:, ["start station latitude", "start station longitude", "start station name"]]

    subset = bike_data[bike_data["hour"] == selected_hour]

    dep_counts = subset.groupby("start station id").count()
    dep_counts = dep_counts.iloc[:, [0]]
    dep_counts.columns= ["Departure Count"]

    arr_counts = subset.groupby("end station id").count()
    arr_counts = arr_counts.iloc[:, [0]]
    arr_counts.columns= ["Arrival Count"]

    trip_counts = dep_counts.join(locations).join(arr_counts)
    trip_counts.head()
    return trip_counts

get_trip_counts(9).head()

,Departure Count,start station latitude,start station longitude,start station name,Arrival Count
start station id,,,,,
72.0,274,40.767272,-73.993929,W 52 St & 11 Ave,283.0
79.0,112,40.719116,-74.006667,Franklin St & W Broadway,222.0
82.0,73,40.711174,-74.000165,St James Pl & Pearl St,45.0
119.0,18,40.696089,-73.978034,Park Ave & St Edwards St,10.0
120.0,79,40.686768,-73.959282,Lexington Ave & Classon Ave,28.0


In [77]:
def plot_station_counts(trip_counts):
    folium_map = folium.Map(location=centre_latlng, zoom_start=13, tiles="CartoDB dark_matter", width='50%')
    
    for index, row in trip_counts.iterrows():
        net_dep = (row["Departure Count"] - row["Arrival Count"])
#         print(net_dep)
        
        pop_up_text = "{st_name}<br> total departures: {dep_count}<br> total arrivals:{arr_count}<br> net departures:{net_dep}"
        pop_up_text = pop_up_text.format(st_name=row["start station name"], dep_count=row["Departure Count"], arr_count=row["Arrival Count"], net_dep=net_dep)
        
        radius = net_dep / 20
        
        if net_dep > 0:
            color="#E37222"
        else:
            color="#0A8A9F"
        
        folium.CircleMarker(location=[row["start station latitude"], row["start station longitude"]], radius=radius, color=color, popup=pop_up_text, fill=True).add_to(folium_map)
    return folium_map

In [78]:
# plot net departures at 6PM

trip_counts = get_trip_counts(18)
folium_map = plot_station_counts(trip_counts)
folium_map

In [79]:
folium_map.save("sample.html")